<a href="https://colab.research.google.com/github/sudharshanpaul/Deep-Learning/blob/main/Auto_Encoders/MovieRecommendation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AutoEncoders

#### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Data Preprocessing

#### Importing the datasets

In [ ]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine='python', encoding='latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine='python', encoding='latin-1')

In [ ]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### Preparing the Training and Test set

In [ ]:
training_set = pd.read_csv('u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')

test_set = pd.read_csv('u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [ ]:
print(f'training_set shape: {training_set.shape}')
print(f'test_set shape: {test_set.shape}')

training_set shape: (79999, 4)
test_set shape: (19999, 4)


#### Getting the number od Users and Movies

In [ ]:
nb_users = int(max(max(training_set[:,0]),max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [ ]:
print(f'Number of Users: {nb_users}')
print(f'Number of Movies: {nb_movies}')

Number of Users: 943
Number of Movies: 1682


#### Converting the data into an array with users in rows and ratings in columns

In [ ]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users+1):
    id_movies = data[:, 1][data[:, 0] == id_users]
    id_ratings = data[:, 2][data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))

  return new_data

In [ ]:
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
training_set[0][:5]

[np.float64(0.0),
 np.float64(3.0),
 np.float64(4.0),
 np.float64(3.0),
 np.float64(3.0)]

#### Converting the data into Torch Tensors

In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Creating the Architecture of Neural Network

In [ ]:
class SAE(nn.Module):

  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)   # Encoding layer 1
    self.fc2 = nn.Linear(20 , 10)         # Encoding Layer 2
    self.fc3 = nn.Linear(10, 20)          # Decoding Layer 1
    self.fc4 = nn.Linear(20, nb_movies)   # Decoding Layer 2
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.activation(self.fc1(x))   # Encoding
    x = self.activation(self.fc2(x))   # Encoding
    x = self.activation(self.fc3(x))   # Decoding
    x = self.fc4(x)                    # Decoding
    return x

In [ ]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

### Training the Stacked AutoEncoder

In [ ]:
nb_epochs = 200
for epoch in range(1, nb_epochs+1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    ## Here we want to create a Batch.. i.e for Batch learning, we have to add a new dimension.., here we are adding ayt 0 index
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.requires_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()  # This method says in which direction we have to update i.e either increasing or decreasing
      train_loss += np.sqrt(loss.item()*mean_corrector)
      s += 1.
      optimizer.step() # This method says the intensity that we have to change
  print(f'epoch: {epoch} and loss: {train_loss/s}')

epoch: 1 and loss: 1.7651047718816144
epoch: 2 and loss: 1.096439468949486
epoch: 3 and loss: 1.053552786730757
epoch: 4 and loss: 1.038220213892997
epoch: 5 and loss: 1.0308594063480443
epoch: 6 and loss: 1.0264175926871542
epoch: 7 and loss: 1.0238905690636644
epoch: 8 and loss: 1.0216788794645555
epoch: 9 and loss: 1.0207257034989399
epoch: 10 and loss: 1.0195438095749463
epoch: 11 and loss: 1.0188955235724169
epoch: 12 and loss: 1.0181791149159598
epoch: 13 and loss: 1.0178175129533138
epoch: 14 and loss: 1.0176081240226558
epoch: 15 and loss: 1.017140220269786
epoch: 16 and loss: 1.0169319057882646
epoch: 17 and loss: 1.0167504311550084
epoch: 18 and loss: 1.016378080856992
epoch: 19 and loss: 1.0163112642773564
epoch: 20 and loss: 1.0160233123758329
epoch: 21 and loss: 1.0157925837212134
epoch: 22 and loss: 1.0158788376059193
epoch: 23 and loss: 1.0158509718554787
epoch: 24 and loss: 1.0158677400157947
epoch: 25 and loss: 1.015373870264434
epoch: 26 and loss: 1.0156599639502388
e

### Testing the SAE

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  ## Here we want to create a Batch.. i.e for Batch learning, we have to add a new dimension.., here we are adding ayt 0 index
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.requires_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.item()*mean_corrector)
    s += 1.
print(f'Test Loss: {test_loss/s}')

Test Loss: 0.9560937891804855
